# Parallel Computation of Preference Matrices

The notebook below exemplifies how `ir_axioms` can be used to compute preference matrices for all runs submitted to the TREC 2020 Deep Learning passage ranking track.

## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.

In [1]:
from sys import modules

if 'google.colab' in modules:
    !pip install -q ir_axioms[examples] python-terrier

We initialize PyTerrier and import all required libraries and load the data from [ir_datasets](https://ir-datasets.com/).

In [2]:
from pyterrier import started, init

if not started():
    init(tqdm="auto")

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Datasets and Index
Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.

In [3]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage/trec-dl-2020/judged"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")

Now define paths where we will store temporary files, datasets, and the search index.

In [4]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)

In [5]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=["text"]
    )

## Submitted Runs
Define the path where you have stored the submitted run files.
(You have to manually download the run files from TREC and store them in a directory of your choice. Then adjust the path below.)

In [6]:
result_dir = Path(
    "/mnt/ceph/storage/data-in-progress/data-research"
    "/web-search/web-search-trec/trec-system-runs"
    "/trec29/deep.passages"
)

We can now load the runs from the run file directory.

In [7]:
from tqdm.auto import tqdm
from pyterrier.io import read_results

run_files = list(result_dir.iterdir())
run_results = [
    read_results(result_file)
    for result_file in tqdm(
        run_files,
        desc="Load runs",
        unit="run",
        total=len(run_files)
    )
]

Load runs:   0%|          | 0/59 [00:00<?, ?run/s]

Concat the retrieved results from all runs in a single data frame.

In [8]:
from pandas import concat

all_results = concat(run_results)
all_results

,qid,docno,rank,score,name
0,3505,1030851,0,-11.050011,TUW-TK-2Layer
1,3505,3998753,1,-11.185551,TUW-TK-2Layer
2,3505,2631212,2,-11.263526,TUW-TK-2Layer
3,3505,329745,3,-11.579689,TUW-TK-2Layer
4,3505,8285654,4,-11.582441,TUW-TK-2Layer
...,...,...,...,...,...
199995,1136962,659447,996,0.006507,p_d2q_rm3_duo
199996,1136962,4101357,997,0.006501,p_d2q_rm3_duo
199997,1136962,7422963,998,0.006494,p_d2q_rm3_duo
199998,1136962,4297877,999,0.006488,p_d2q_rm3_duo


## Import Axioms
Here we're listing which axioms we want to compute preferences for.
Because some axioms require API calls or are computationally expensive, we cache all axioms using `ir_axiom`'s tilde operator (`~`).

In [9]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC, STMC1, STMC1_f, STMC2, STMC2_f, LNC1, TF_LNC, LB1,
    REG, ANTI_REG, ASPECT_REG, REG_f, ANTI_REG_f, ASPECT_REG_f
)

axioms = [
    ArgUC(), QTArg(), QTPArg(), aSL(),
    LNC1(), TF_LNC(), LB1(),
    PROX1(), PROX2(), PROX3(), PROX4(), PROX5(),
    REG(), REG_f(), ANTI_REG(), ANTI_REG_f(), ASPECT_REG(), ASPECT_REG_f(),
    AND(), LEN_AND(), M_AND(), LEN_M_AND(), DIV(), LEN_DIV(),
    RS_TF(), RS_TF_IDF(), RS_BM25(), RS_PL2(), RS_QL(),
    TFC1(), TFC3(), M_TDC(), LEN_M_TDC(),
    STMC1(), STMC1_f(), STMC2(), STMC2_f(),
]
axioms_cached = [~axiom for axiom in axioms]
axiom_names = [axiom.name for axiom in axioms]

## Preference Computation
After having defined the axioms to compute, we create a new PyTerrier pipeline that computes preference matrices for the top-10 results of each system.

In [10]:
from pyterrier import Transformer
from ir_axioms.backend.pyterrier.transformers import AxiomaticPreferences

compute_preferences = Transformer.from_df(all_results) % 10 >> AxiomaticPreferences(
    axioms=axioms,
    # axioms=axioms_cached,
    axiom_names=axiom_names,
    index=index_dir,
    dataset=dataset_name,
    cache_dir=cache_dir,
    verbose=True,
)

To speed up computation, let's distribute preference matrix computation across 4 cores.

In [11]:
compute_preferences = compute_preferences.parallel(4)

In the next step, we parallely compute the preference matrices (line 5)
and measure the elapsed time.

In [12]:
from time import perf_counter_ns

time = perf_counter_ns()

preferences = compute_preferences.transform(dataset.get_topics())

elapsed_time = perf_counter_ns() - time
elapsed_time_seconds = elapsed_time / 1_000_000_000
print(f"Elapsed time: {elapsed_time_seconds:.2f}s")
preferences_per_second = len(preferences) / elapsed_time_seconds
print(f"Preferences per second: {preferences_per_second:.2f}s")

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
Computing query axiom preferences: 100%|██████████| 826/826 [06:54<00:00,  2.00query/s]


Elapsed time: 445.94s
Preferences per second: 601.71s


Here's the resulting Pandas `DataFrame` containing all preferences:

In [13]:
preferences

,qid,query,docno_a,rank_a,score_a,name,docno_b,rank_b,score_b,ArgUC_preference,...,RS-PL2_preference,RS-QL_preference,TFC1_preference,TFC3_preference,M-TDC_preference,LEN-M-TDC_preference,STMC1_preference,STMC1-fastText_preference,STMC2_preference,STMC2-fastText_preference
0,1030303,who is aziz hashim,8726437,0,-6.537941,TUW-TK-2Layer,8726437,0,-6.537941,0,...,0,0,0,0,0,0,0,0,0,1
1,1030303,who is aziz hashim,8726437,0,-6.537941,TUW-TK-2Layer,8726435,1,-6.964480,0,...,-1,-1,0,0,0,0,1,-1,0,0
2,1030303,who is aziz hashim,8726437,0,-6.537941,TUW-TK-2Layer,5681381,2,-7.851367,0,...,1,1,0,0,0,0,1,-1,0,1
3,1030303,who is aziz hashim,8726437,0,-6.537941,TUW-TK-2Layer,7284047,3,-8.081900,0,...,-1,1,0,0,0,0,-1,-1,0,0
4,1030303,who is aziz hashim,8726437,0,-6.537941,TUW-TK-2Layer,8726433,4,-8.227568,0,...,-1,-1,0,0,0,0,1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59623,997622,where is the show shameless filmed,4518222,9,78.546576,p_d2q_rm3_duo,73462,5,81.672453,0,...,1,1,0,0,0,0,1,1,0,0
59624,997622,where is the show shameless filmed,4518222,9,78.546576,p_d2q_rm3_duo,73460,6,81.004847,0,...,-1,1,0,0,0,0,-1,-1,0,0
59625,997622,where is the show shameless filmed,4518222,9,78.546576,p_d2q_rm3_duo,4643397,7,80.117589,0,...,-1,1,0,0,0,0,-1,-1,0,0
59626,997622,where is the show shameless filmed,4518222,9,78.546576,p_d2q_rm3_duo,73465,8,79.295991,0,...,-1,-1,-1,0,0,0,-1,-1,0,0


## Note About Backends
The parallelization is implemented in [PyTerrier](https://pyterrier.readthedocs.io/en/latest/parallel.html).
In this example we use the default [Joblib](https://joblib.readthedocs.io/en/latest/) backend which splits computation per query and runs on multiple cores on the same machine.

However, you could also use the [Ray](https://www.ray.io/) backend.
With Ray you can connect to remote clusters and distribute the workload across multiple machines
(e.g. [Kubernetes](https://docs.ray.io/en/latest/cluster/kubernetes.html), [Hadoop/Spark](https://docs.ray.io/en/latest/cluster/yarn.html), or [Slurm](https://docs.ray.io/en/latest/cluster/slurm.html)).
Please refer to the [Ray documentation](https://docs.ray.io/en/latest/) for detailed instructions on how to connect your cluster.